In [ ]:
# Not sure why the output isnt visble had made this project seperately on jypyter notebook,Feel free to check out with outputs on my github : https://bit.ly/3jbLDaw


#import libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn import neighbors, linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from collections import Counter
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## DOMAIN: Healthcare

**Import and merge data:**

In [ ]:
n= pd.read_csv('/kaggle/input/healthcare-insurance/Part1 - Normal.csv')
h= pd.read_csv('/kaggle/input/healthcare-insurance/Part1 - Type_H.csv')
s= pd.read_csv('/kaggle/input/healthcare-insurance/Part1 - Type_S.csv')

**Remove duplicates**

In [ ]:
print("Duplicate labels found in dataset:")
print(n.Class.unique(),h.Class.unique(),s.Class.unique())

In [ ]:
h['Class'] =h['Class'].str.replace('type_h','Type_H')
n['Class'] =n['Class'].str.replace('Nrmal','Normal')
s['Class'] =s['Class'].str.replace('tp_s','Type_S')
print("Adjusting dataset to have appropriate class:")
print(n.Class.unique(),h.Class.unique(),s.Class.unique())

In [ ]:
#reformatting for visualisation
h['Class'] =h['Class'].str.replace('Type_H','Type H')
s['Class'] =s['Class'].str.replace('Type_S','Type S')
print(n.Class.unique(),h.Class.unique(),s.Class.unique())

## Creating a function to understand each dataset independently

In [ ]:
def cleansing (x):
    
    print("\033[1m HEAD\n\033[0m",x.head())
    print("\033[1m\nTAIL\n\033[0m",x.tail())
    print("\033[1m\nDATA TYPES\n\033[0m",s.dtypes)
    print("\033[1m\nDESCRIBE\n\033[0m",x.describe())
    print("\033[1m\nMissing value is:\033[0m",x.isnull().values.any())
    print("\033[1m\nDupicate value is:\033[0m",x.duplicated().values.any())
    print("\033[1m\nShape of Dataset:\033[0m",x.shape)
    print("\033[1m\nSize of Dataset:\033[0m",x.size)
    print("\033[1m\nDimension of Dataset:\033[0m",x.ndim)  

In [ ]:
def correlate(x):
    cl=x.Class.nunique()
    if cl>2:
        cls="Mixed"
    else:
        cls=x.Class[0] 
    print(f"\033[1m\nDiagonal Correlation Heat Map Type: {cls}\033[0m\n")
    corr = x.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    
    ax = sns.heatmap(
        corr,mask=mask, 
        vmin=-1,vmax=1, center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True,linewidths=.5, cbar_kws={"shrink": .5})
   
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right');

In [ ]:
def distribution(x):
    cl=x.Class.nunique()
    if cl>2:
        cls="Mixed"
    else:
        cls=x.Class[0]  
    
    print(f"\033[1m\nDistribution Observed In Dataset Type: {cls}\033[0m\n")
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True, figsize=(12,6));
    sns.histplot( x["P_incidence"] , color="skyblue", ax=ax1)
    ax1.set_title("Incidence");
    sns.histplot( x["P_tilt"] , color="dimgray", ax=ax2);
    ax2.set_title("Tilt");
    sns.histplot( x["L_angle"] , color="olive", ax=ax3);
    ax3.set_title("Angle");
    sns.histplot( x["S_slope"] , color="gold", ax=ax4);
    ax4.set_title("Slope");
    sns.histplot( x["P_radius"] , color="teal", ax=ax5);
    ax5.set_title("Radius");
    sns.histplot( x["S_Degree"] , color="seagreen", ax=ax6);
    ax6.set_title("Degree");
    
    plt.tight_layout();

## Analysing Dataset :Type N

In [ ]:
cleansing(n)

In [ ]:
distribution(n)

In [ ]:
correlate(n)

## Analysing Dataset :Type H

In [ ]:
cleansing(h)

In [ ]:
distribution(h)

In [ ]:
correlate(h)

## Analysing Dataset :Type S

In [ ]:
cleansing(s)

In [ ]:
distribution(s)

In [ ]:
correlate(s)

## Merging All Datasets 

In [ ]:
c = pd.concat(([n,h,s]), ignore_index=True)
c

**Visually confirming successful combination of all 3 datasets into 1**

In [ ]:
c.Class.value_counts().plot(kind = 'bar');
plt.axhline(y=(n.Class.count()), color='r', linestyle='dotted');
plt.axhline(y=(h.Class.count()), color='r', linestyle='dotted');
plt.axhline(y=(s.Class.count()), color='r', linestyle='dotted');

The number of type s is more than double of type h

## Using the same defined function to analyse and label the new combined data.

In [ ]:
cleansing(c)

**INFERENCE<br>
-No empty row in dataset from count<br>
-Very high standard deviation fir column S_Degree<br>
-Max value of S_Degree column confirms something unusual which needs to be further checked<br>
-The huge gap between 75% and max value of S_Degree indicates some outliers with unusally high value**

In [ ]:
distribution(c)

**Outlier visualised in S_Degree**

In [ ]:
correlate(c)

## Overall co-relation of new combined dataset 

In [ ]:
sns.pairplot(c,hue="Class")
plt.tight_layout();

**from pairplot it seems like type h and type s patients are opposite to each other
for example:type h maybe healthy patients and type s maybe sick.....checking hypothesis by comparing distribution of type h and type s**

In [ ]:
hs = pd.concat(([h,s]), ignore_index=True)
sns.pairplot(hs,hue="Class");

**As we can see most data would be easily classified using this dataset**

In [ ]:
sns.displot(data=c,x="S_slope", hue="Class",kind="kde", height=6,multiple="fill", clip=(0, None),
            palette="ch:rot=-.25,hue=1,light=.75");

**High values of S_slope (Above 80) gets classified as Type S**

In [ ]:
sns.displot(data=c,x="P_radius", hue="Class",kind="kde", height=6,multiple="fill", clip=(0, None),
            palette="ch:rot=-.25,hue=1,light=.75");

In [ ]:
sns.displot(data=c,x="S_Degree", hue="Class",kind="kde", height=6,multiple="fill", clip=(0, None),
            palette="ch:rot=-.25,hue=1,light=.75");

**Outliers might be present here but a direct correlation can be assumed between S_Degree and Type S Classification**

## Identifying outlier

In [ ]:
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16}
sns.boxplot(c["S_Degree"]);
out=int(c["S_Degree"].max())
plt.axvline(x=([c["S_Degree"].max()]), color='r', linestyle='dotted');
plt.text(out,.6, out,fontdict=font );

**Dropping outlier row with very high max value**

In [ ]:
q = c["S_Degree"].max()
c1=c[c["S_Degree"] < q]
sns.boxplot(c1["S_Degree"]);

In [ ]:
sns.displot(data=c1,x="S_Degree", hue="Class",kind="kde", height=6,multiple="fill", clip=(0, None),
            palette="ch:rot=-.25,hue=1,light=.75");

**INFERENCE<br>
Even after the very high outlier value a direct correlation can be seen between high values of S_Degree and Type S Classification<br>
Confirming the same with bar plot**

In [ ]:
sns.barplot(x='Class', y='S_Degree', data=c1,palette ='magma');
plt.title('Type S patients highly correlated to S_Degree values');

In [ ]:
distribution(c1)

## EDA 

**Comparing Type H and Type S as they are visually easier to decipher without the normal classfication<br>
which is always overlapping between these two classifications<br>
For example consider joint plot with combined dataset**

In [ ]:
sns.set_theme(style="ticks")
sns.jointplot(data=c,x="S_slope", y="P_radius", hue="Class",kind="kde");

**Same jointplot without the set of people from normal classification**

In [ ]:
sns.set_theme(style="ticks")
plot=sns.jointplot(data=hs,x="S_slope", y="P_radius", hue="Class",kind="kde");
#setting equal magnitude of x and y to get a 1:1 scaled visualisation
plot.ax_marg_x.set_xlim(0, 120)
plot.ax_marg_y.set_ylim(60, 180);

**INFERENCE<br>
P_ Radius : Beyond a threshold of deviation will get patient classifeid from Type H to Type S<br>
S_Slope is directly prroportional i.e a higher value of S_slope will tend to be classified as Type S**

In [ ]:
sns.jointplot(data=hs,x="P_incidence", y="L_angle", hue="Class",kind="kde");

**INFERENCE<br>
P_incidence and L_angle have a positive correlation w.r.t each other<br>
If both are low : Type H<br>
If both are high: Type S**

In [ ]:
plot=sns.jointplot(data=hs,x="P_incidence", y="P_tilt", hue="Class",kind="kde");
#setting equal magnitude of x and y to get a 1:1 scaled visualisation
plot.ax_marg_x.set_xlim(0, 140);
plot.ax_marg_y.set_ylim(-40,100);

In [ ]:
plot=sns.jointplot(data=hs,x="P_radius", y="L_angle", hue="Class",kind="kde");
#setting equal magnitude of x and y to get a 1:1 scaled visualisation
plot.ax_marg_x.set_xlim(40,180 )
plot.ax_marg_y.set_ylim(0, 140);

**INFERENCE<br>
Higher L_angle values tend to be of Type S Classification**

In [ ]:
c1.corr()

**INFERENCE<br>
-No coorelation between S_Degree and P_radius<br>
-Strong coorealtion between S_slope and P_incidence ~80%<br>
-Similiar strong relation was showed visually earlier between P_incidence and L_angle of ~74%**

In [ ]:
plt.figure(figsize=(6,3))
c1.Class.hist();

**There is a slight class imbalance which we may need to resolve if it affects our prediction.<br>
For now have made stratify=y in train_test_split so that the model can try to accurately map the classification**

## Model training, testing and tuning

In [ ]:
sc_X = StandardScaler()
X =  pd.DataFrame(sc_X.fit_transform(c1.drop(["Class"],axis = 1),),
        columns=['P_incidence', 'P_tilt', 'L_angle', 'S_slope', 'P_radius','S_Degree'])
y = c1.Class
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1,stratify=y)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn_model = knn.fit(X_train, y_train,)

In [ ]:
y_true, y_pred = y_test, knn_model.predict(X_test)
print('k-NN score for training set: %f' % knn_model.score(X_train, y_train))
print('k-NN score for test set: %f' % knn_model.score(X_test, y_test))
print(classification_report(y_true, y_pred))

Precision - Precision is the ratio of correctly predicted positive observations to the total predicted positive observations. High precision relates to the low false positive rate. Our precision per class is varying due to imbalance in dataset.A weighted average gives a better overall reading, BUt since it is a medical dataset we would like this to go up.

Recall is the ratio of correctly predicted positive observations to the all observations in actual class - yes. For a healthcare dataset its is essential that recall is prioritised and we can get as high a value for recall as possible.

F1 Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. F1 is usually more useful than accuracy, especially in an uneven class distribution. Accuracy works best if false positives and false negatives have similar cost. If the cost of false positives and false negatives are very different, it’s better to look at both Precision and Recall


## Find optimal value of k

In [ ]:
# creating odd list of K for KNN
myList = list(range(3,20))

# subsetting just the odd ones
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold accuracy scores
ac_scores = []

for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    # predict the response
    y_pred = knn.predict(X_test)
    # evaluate accuracy
    scores = accuracy_score(y_test, y_pred)
    ac_scores.append(scores)
              
# changing to misclassification error
MSE = [1 - x for x in ac_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
## plot misclassification error vs k
plt.figure(figsize=(6,3))
plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.axvline(x=(optimal_k), color='r', linestyle='dotted');
plt.text(optimal_k,.25, optimal_k,fontdict=font );

## Using best value for k in model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=optimal_k)
knn_model = knn.fit(X_train, y_train,)
print('k-NN score for training set: %f' % knn_model.score(X_train, y_train))
print('k-NN score for test set: %f' % knn_model.score(X_test, y_test))
print(classification_report(y_true, y_pred))

**Using best k value has made a significant change.<br>
Model is predicting better, it is no longer overfitting.Infact test scores are now higher that training scores.<br>
Weighted average scores for precision and recall has increased to almost 80 percent.<br>
But,our individual classes are imbalanced, lets check if blancing input deatures will help in getting higher accuracy**

## Using SMOTE to adjust the class imbalance

In [ ]:
oversample = SMOTE()
Xs_train,ys_train = oversample.fit_resample(X_train, y_train)

In [ ]:
Xs_test,ys_test = oversample.fit_resample(X_test, y_test)

In [ ]:
counter = Counter(ys_train)
print(counter)

In [ ]:
# creating odd list of K for KNN
myList = list(range(3,25))

# subsetting just the odd ones
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold accuracy scores
ac_scores = []

for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(Xs_train, ys_train)
    # predict the response
    ys_pred = knn.predict(Xs_test)
    # evaluate accuracy
    scores = accuracy_score(ys_test, ys_pred)
    ac_scores.append(scores)
              
# changing to misclassification error
MSE = [1 - x for x in ac_scores]

# determining best k
optimal_ks = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_ks)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=optimal_ks)
smote_model=knn.fit(Xs_train, ys_train)

ys_true, ys_pred = ys_test, knn_model.predict(Xs_test)
print('k-NN score for training set: %f' % smote_model.score(Xs_train, ys_train))
print('k-NN score for test set: %f' % smote_model.score(Xs_test, ys_test))
print(classification_report(ys_true, ys_pred))

**Precision for classifying minor classes have slightly increased but the F1 scores have reduced.<br>
While a higher precision/recall is assumed to be a better way to judge this model, this cannot necessarily come at the expense of lower accuracy.**

In [ ]:
#import GridSearchCV
from sklearn.model_selection import GridSearchCV
#In case of classifier like knn the parameter to be tuned is n_neighbors
param_grid = {'n_neighbors':np.arange(1,50)}
knn = KNeighborsClassifier()
knn_cv= GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(X,y)

print("Best Score:" + str(knn_cv.best_score_))
print("Best Parameters: " + str(knn_cv.best_params_))


Gridsearch also does not help refining result, so we will follow the model using best n value

In [ ]:
# Display confusion matrix
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
df_conf_mat = pd.DataFrame(conf_mat)
plt.figure(figsize = (10,7))
sns.heatmap(df_conf_mat, annot=True,cmap='Blues', fmt='g')

**Conclusion**

While the true positive rate and true negative rate is good there are quite a few misclassification for minority data.<br>
As we saw from bivariate and multivariate analysis differentiating between 'Type H' and 'Type S' is easier from the data availabe.<br>
To Classify Type N accurately in between the other 2 conditions we will need better features which associate with the class.<br>
Knn might not be the best approach maybe taking data to a different dimension like svm will be useful to cluster better.
